In [ ]:
path = '/kaggle/input/ithack/Jan_2024-Nov_2024.csv'

In [ ]:
import pandas as pd
df = pd.read_csv(path)

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.isna().sum() / len(df) * 100

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.shape

In [ ]:
df.head().to_dict()

In [ ]:
df.info()

In [ ]:
timestamp_cols = [
    'LAST_POOL_ACTIVITY_TIMESTAMP',
    'FIRST_POOL_ACTIVITY_TIMESTAMP',
    'LAST_SWAP_TIMESTAMP'
]

for col in timestamp_cols:
    df[col] = df[col].str.split('.').str[0]
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S', errors='coerce')


In [ ]:
df.info()

In [ ]:
datetime_cols = df.select_dtypes(include=['datetime64[ns]']).columns.tolist()

for col in datetime_cols:
    df[f"{col}_ts"] = df[col].astype('int64') // 10**9  # Giây

In [ ]:
for col in datetime_cols:
    df[f'{col}_hour'] = df[col].dt.hour
    df[f'{col}_day'] = df[col].dt.day
    df[f'{col}_weekday'] = df[col].dt.weekday  # Monday = 0, Sunday = 6
    df[f'{col}_month'] = df[col].dt.month

In [ ]:
df.isna().sum() / len(df) * 100

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.drop(columns=datetime_cols, inplace=True)

In [ ]:
numeric_cols = df.select_dtypes(include=['float64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

In [ ]:
numeric_cols

In [ ]:
categorical_cols

In [ ]:
for col in df.columns:
    print(f"{col}: {df[col].dtype}")

In [ ]:
datetime_cols = df.select_dtypes(include=['datetime64[ns]']).columns.tolist()

for col in datetime_cols:
    df[f"{col}_ts"] = df[col].astype('int64') // 10**9  # Giây


In [ ]:
for col in datetime_cols:
    df[f'{col}_hour'] = df[col].dt.hour
    df[f'{col}_day'] = df[col].dt.day
    df[f'{col}_weekday'] = df[col].dt.weekday  # Monday = 0, Sunday = 6
    df[f'{col}_month'] = df[col].dt.month

In [ ]:
df.shape

In [ ]:
df.drop(columns=datetime_cols, inplace=True)

In [ ]:
df.info()

In [ ]:
numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
numeric_cols

In [ ]:
time_cols = ['LAST_SWAP_TIMESTAMP_hour', 'LAST_SWAP_TIMESTAMP_day', 
             'LAST_SWAP_TIMESTAMP_weekday', 'LAST_SWAP_TIMESTAMP_month']

df[time_cols] = df[time_cols].fillna(-1)

In [ ]:
df.isna().sum() / len(df) * 100

In [ ]:
df = df.drop(columns=['LAST_SWAP_TX_ID'])

In [ ]:
numeric_with_nan = [
    col for col in df.select_dtypes(include=['number']).columns 
    if df[col].isnull().any()
]
numeric_with_nan

In [ ]:
# Plot histogram for each numeric column with NaNs
for col in numeric_with_nan:
    plt.figure(figsize=(6, 4))
    plt.hist(df[col].dropna(), bins=50)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.show()

In [ ]:
col_to_fill_0 = ['TOTAL_ADDED_LIQUIDITY',                    
'TOTAL_REMOVED_LIQUIDITY',                  
'NUM_LIQUIDITY_ADDS',                       
'NUM_LIQUIDITY_REMOVES',                    
'ADD_TO_REMOVE_RATIO']
df[col_to_fill_0] = df[col_to_fill_0].fillna(0)

In [ ]:
df.dropna(subset=['MINT'], inplace=True)

In [ ]:
# Danh sách 8 cột đặc trưng thời gian suy ra
time_derived_features_to_fill = [
    'LAST_POOL_ACTIVITY_TIMESTAMP_hour',
    'LAST_POOL_ACTIVITY_TIMESTAMP_day',
    'LAST_POOL_ACTIVITY_TIMESTAMP_weekday',
    'LAST_POOL_ACTIVITY_TIMESTAMP_month',
    'FIRST_POOL_ACTIVITY_TIMESTAMP_hour',
    'FIRST_POOL_ACTIVITY_TIMESTAMP_day',
    'FIRST_POOL_ACTIVITY_TIMESTAMP_weekday',
    'FIRST_POOL_ACTIVITY_TIMESTAMP_month'
]

for col in time_derived_features_to_fill:
    if col in df.columns:
        df[col].fillna(-1, inplace=True)

print("\nSố lượng NaN ở các cột thời gian suy ra sau khi điền:")
print(df[time_derived_features_to_fill].isnull().sum())

In [ ]:
df.isna().sum()/len(df)*100

In [ ]:
inactivity_status_one_hot = pd.get_dummies(df['INACTIVITY_STATUS'], prefix='INACTIVITY_STATUS')


df = pd.concat([df, inactivity_status_one_hot], axis=1)

df.drop('INACTIVITY_STATUS', axis=1, inplace=True)

In [ ]:
estimated_contamination = 0.005 

In [ ]:
import optuna
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split 

def objective(trial, X_data):

    # Gợi ý các tham số cho Isolation Forest
    n_estimators = trial.suggest_int('n_estimators', 50, 300) # Số cây trong rừng
    max_samples = trial.suggest_int('max_samples', 100, min(2048, len(X_data))) # Số mẫu lấy để xây cây
    
    model = IsolationForest(n_estimators=n_estimators,
                            max_samples=max_samples,
                            contamination=estimated_contamination, # Sử dụng giá trị ước tính
                            random_state=trial.number) # Sử dụng trial.number để đa dạng hóa seed

    # Fit mô hình với toàn bộ tập dữ liệu đặc trưng
    model.fit(X_data)

    # Tính điểm bất thường
    anomaly_scores = model.decision_function(X_data)

    # Phân loại điểm thành "inliers" (điểm > 0) và "outliers" (điểm <= 0) dựa trên ngưỡng 0
    # (trước khi áp dụng ngưỡng contamination)
    inlier_scores = anomaly_scores[anomaly_scores > 0]
    outlier_scores = anomaly_scores[anomaly_scores <= 0]

    # Tính điểm trung vị cho từng nhóm
    # Xử lý trường hợp một trong hai nhóm rỗng
    median_inlier = np.median(inlier_scores) if len(inlier_scores) > 0 else 0
    median_outlier = np.median(outlier_scores) if len(outlier_scores) > 0 else 0

    # Hàm mục tiêu: Tối đa hóa sự khác biệt giữa trung vị inlier và trung vị outlier
    # Điểm bất thường càng âm càng tốt, nên median_outlier sẽ là một số âm.
    # median_inlier - median_outlier = median_inlier + abs(median_outlier) -> muốn giá trị này lớn
    objective_value = median_inlier - median_outlier

    # Optuna mặc định tìm cách minimize. Để tối đa hóa, ta trả về giá trị âm
    return -objective_value

In [ ]:
columns_to_drop_for_X = [
    'LIQUIDITY_POOL_ADDRESS',
    'MINT', 
    'LAST_SWAP_TX_ID', 
    'LAST_POOL_ACTIVITY_TIMESTAMP_ts',
    'FIRST_POOL_ACTIVITY_TIMESTAMP_ts',
    'LAST_SWAP_TIMESTAMP_ts'
]
X = df.drop(columns=columns_to_drop_for_X, errors='ignore')

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X), n_trials=20) 
# In ra kết quả tốt nhất
print("\nKết quả tối ưu hóa:")
print(f"Tham số tốt nhất: {study.best_params}")
print(f"Giá trị hàm mục tiêu tốt nhất (âm của sự khác biệt trung vị): {study.best_value}")

In [ ]:
best_params = study.best_params
best_model = IsolationForest(n_estimators=best_params['n_estimators'],
                             max_samples=best_params['max_samples'],
                             contamination=estimated_contamination, 
                             random_state=42) 

# Fit mô hình tốt nhất với toàn bộ dữ liệu để sử dụng
best_model.fit(X)

In [ ]:
# Bây giờ bạn có thể sử dụng best_model để tính điểm bất thường và dự đoán nhãn
anomaly_scores_tuned = best_model.decision_function(X)
predictions_tuned = best_model.predict(X)

In [ ]:
df['anomaly_score_isolation_forest'] = anomaly_scores_tuned
df['is_outlier_isolation_forest'] = predictions_tuned

In [ ]:
# Xem 10 hàng có điểm bất thường thấp nhất (bất thường nhất)
top_anomalies = df.sort_values(by='anomaly_score_isolation_forest').head(10)
print("\nTop 10 ứng viên Rug Pull dựa trên điểm bất thường:")
print(top_anomalies)

# Xem tất cả các hàng được dự đoán là outliers (-1)
predicted_outliers_df = df[df['is_outlier_isolation_forest'] == -1]
print(f"\nSố lượng điểm được dự đoán là outliers: {len(predicted_outliers_df)}")
print("Thông tin về các điểm được dự đoán là outliers:")
print(predicted_outliers_df.head())

In [ ]:
anomaly_candidates_df = df[df['is_outlier_isolation_forest'] == -1]
anomaly_candidates_df.head(1).to_dict()

In [ ]:
subset_to_label = df.sort_values(by='anomaly_score_isolation_forest').head(478)
indices_to_label = subset_to_label.index

In [ ]:
df['true_label'] = np.nan

In [ ]:
import joblib
import os 
output_dir = '/kaggle/working/'

model_filename = 'isolation_forest_model.joblib'

# Tạo đường dẫn đầy đủ để lưu file
output_path = os.path.join(output_dir, model_filename)

print(f"Đang lưu mô hình tới: {output_path}")
joblib.dump(best_model, output_path)
print("Đã lưu mô hình thành công.")

# Sau khi code này chạy xong, file 'isolation_forest_model.joblib' sẽ có trong thư mục /kaggle/working/

In [ ]:
subset_to_label.head(10).to_dict()

In [ ]:
for i, row in subset_to_label.head(30).iterrows():
    print(f"Index: {i}, Data: {row.to_dict()}")

In [ ]:
subset_to_label.to_csv('/kaggle/working/subset_to_label.csv', index=False)